In [3]:
"""
Configuración inicial del entorno de entrenamiento
─────────────────────────────────────────────────────────────
Esta celda importa todas las librerías necesarias para:
 - Procesamiento de imágenes (OpenCV, PIL, tifffile, matplotlib)
 - Entrenamiento de modelos (PyTorch, optimizadores, métricas)
 - Visualización de resultados (seaborn, matplotlib)
 - Carga de modelos personalizados (TransformerObjectDetection)
 - Utilidades de detección y conteo (utils.counting)
 - Manejo de rutas, archivos y estructuras de carpetas
"""

# ------------------------------------------------------------
# Librerías del sistema y utilidades generales
# ------------------------------------------------------------
import os                 # Manejo del sistema de archivos y variables de entorno
import glob               # Búsqueda de archivos por patrones
import pathlib            # Alternativa orientada a objetos para manejo de rutas
from pathlib import Path  # Clase Path para rutas multiplataforma
import math               # Funciones matemáticas básicas
import random             # Generación de números aleatorios
import time               # Medición de tiempos de ejecución

# ------------------------------------------------------------
# Librerías científicas y de manejo de datos
# ------------------------------------------------------------
import numpy as np               # Cálculo numérico eficiente con arreglos
import pandas as pd              # Manipulación de datos tabulares
import seaborn as sns            # Visualizaciones estadísticas (heatmaps, etc.)

# ------------------------------------------------------------
# Librerías de visualización
# ------------------------------------------------------------
import matplotlib.pyplot as plt             # Gráficos y figuras
import matplotlib.patches as patches         # Dibujos sobre imágenes (rectángulos, etc.)

# ------------------------------------------------------------
# Librerías para procesamiento de imágenes
# ------------------------------------------------------------
from PIL import Image, ImageDraw  # Manipulación de imágenes (dibujar, abrir, guardar)
from tifffile import imread       # Lectura de archivos TIFF
import cv2                        # Procesamiento de imágenes en formato OpenCV

# ------------------------------------------------------------
# Librerías de Deep Learning
# ------------------------------------------------------------
import torch
import torch.optim as optim
import torchvision.transforms as T  # Transformaciones de datos (rotar, normalizar, etc.)

# ------------------------------------------------------------
# Modelos y utilidades personalizadas del proyecto
# ------------------------------------------------------------
from model.transformer import TransformerObjectDetection  # Modelo Transformer personalizado
from model.trainer import trainer                         # Función de entrenamiento principal
import utils.counting                                     # Funciones auxiliares de inferencia
from dstadam import DSTAdam                               # Optimizador especializado (Adam variante)

# ------------------------------------------------------------
# Modelos adicionales de Ultralytics (YOLO, RT-DETR)
# ------------------------------------------------------------
from ultralytics import YOLO     # Modelo YOLO para detección de objetos
from ultralytics import RTDETR   # Modelo RT-DETR (transformer basado en detección rápida)

# ------------------------------------------------------------
# Métricas de evaluación
# ------------------------------------------------------------
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve,
    confusion_matrix, ConfusionMatrixDisplay
)


C:\Users\BSG\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
"""
Definición de rutas de trabajo
─────────────────────────────────────────────
En esta celda se configuran las rutas principales del proyecto:
 - Carpeta base y subcarpetas de imágenes y etiquetas.
 - Directorios de entrenamiento y modelos.
 - Se listan los archivos disponibles para verificar integridad.
"""

# ================================================================
# Definición de rutas base
# ================================================================

# Directorio raíz del proyecto (donde se ejecuta el notebook)
BASE_DIR = Path(os.getcwd())

# Directorios de imágenes y etiquetas de entrenamiento
IMAGES_DIR = BASE_DIR / "Zones_cbbox_dataset_10/train/images"
LABELS_DIR = BASE_DIR / "Zones_cbbox_dataset_10/train/labels"

# Directorios de dataset completo y almacenamiento de modelos
TRAIN_DIR = BASE_DIR / "Zones_cbbox_dataset_10/"
MODEL_DIR = BASE_DIR / "models_finetuned/bit_sted/"

# ================================================================
# Exploración de archivos
# ================================================================

# Obtiene la lista ordenada de archivos .tif (imágenes) y .txt (etiquetas)
imagenes = sorted(IMAGES_DIR.glob("*.tif"))
etiquetas = sorted(LABELS_DIR.glob("*.txt"))

# Muestra la cantidad de imágenes y etiquetas encontradas
print(f"Total de imágenes TIF: {len(imagenes)}")
print(f"Total de etiquetas TXT: {len(etiquetas)}")



Total de imágenes TIF: 2440
Total de etiquetas TXT: 2440


In [6]:
"""
Configuración de parámetros de entrenamiento
─────────────────────────────────────────────
Esta celda define los hiperparámetros y ajustes generales del proceso
de entrenamiento, incluyendo tamaño de lote, número de épocas,
ubicación de datos y dispositivo de cómputo (CPU o GPU).
"""

# ================================================================
# Parámetros principales del entrenamiento
# ================================================================

batch_size = 32          # Número de imágenes por lote (batch)
epochs = 10              # Número total de épocas de entrenamiento

# Directorios de datos y destino para guardar modelos
folder_data = str(TRAIN_DIR) + "/"   # Carpeta raíz del dataset
save = str(MODEL_DIR) + "/"          # Carpeta donde se guardarán los modelos

# Número total de imágenes disponibles en el conjunto de entrenamiento
train_size = len(sorted(IMAGES_DIR.glob("*.tif")))

# Selección automática del dispositivo: GPU si está disponible, de lo contrario CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cálculo del número total de iteraciones (batches * épocas)
iters = math.ceil(train_size / batch_size) * epochs


In [7]:
"""
Inicialización del modelo, optimizador y programador de tasa de aprendizaje
─────────────────────────────────────────────────────────────
En esta celda se crea la instancia del modelo TransformerObjectDetection,
se configura el optimizador DSTAdam y se define el programador (scheduler)
para ajustar dinámicamente la tasa de aprendizaje durante el entrenamiento.
"""

# ================================================================
# Definición del modelo Transformer personalizado
# ================================================================
model = TransformerObjectDetection(
    image_size=224,       # Tamaño de entrada del modelo (224×224 píxeles)
    N_channels=3,         # Número de canales de entrada (RGB)
    n_model=512,          # Dimensión del espacio de características internas
    num_blks=2,           # Número de bloques del transformador
    device=device,        # Dispositivo de cómputo (CPU o GPU)
    bitNet=True,          # Activación de arquitectura binarizada (Bit-STED)
    alpha=1.0,            # Parámetro de regularización o escala interna
    gamma=1.5             # Factor de ajuste para la función de pérdida
)

# Enviar el modelo al dispositivo seleccionado
model.to(device)

# ================================================================
# Definición del optimizador
# ================================================================
optimizer = DSTAdam(
    model.parameters(),   # Parámetros del modelo a optimizar
    iters=iters           # Número total de iteraciones (control de tasa adaptativa)
)

# ================================================================
# Definición del programador de tasa de aprendizaje
# ================================================================
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',           # Reduce la LR cuando la métrica objetivo (pérdida) deja de mejorar
    factor=0.50,          # Multiplicador de reducción (reduce LR al 50%)
    patience=6            # Número de épocas sin mejora antes de reducir la LR
)

In [9]:
"""
Ejecución del entrenamiento del modelo
─────────────────────────────────────────────
En esta celda se llama a la función `trainer`, que ejecuta el ciclo
completo de entrenamiento del modelo TransformerObjectDetection.
Se incluyen los parámetros principales como el optimizador, el
scheduler y las rutas para guardar los resultados.
"""

# ================================================================
# Ejecución del proceso de entrenamiento
# ================================================================

trainer(
    model,                  # Modelo a entrenar (instancia de TransformerObjectDetection)
    folder_data,            # Ruta base del dataset (imágenes y etiquetas)
    None,                   # Parámetro reservado (por ejemplo, validación o dataset extra)
    optimizer,              # Optimizador definido previamente (DSTAdam)
    scheduler,              # Programador de tasa de aprendizaje (ReduceLROnPlateau)
    epochs=epochs,          # Número de épocas de entrenamiento
    use_scheduler_dict=True,# Habilita el uso del scheduler dinámico
    device=device,          # Dispositivo de cómputo (CPU o GPU)
    folder_save_results=save,# Carpeta donde se guardarán resultados y pesos
    batch_size=batch_size   # Tamaño del lote por iteración
)


Weights normal init applied to model
The model has 6,622,088 trainable parameters
The model size: 26.494MB
Load train data with 2440 images
Load val data with 1377 images
Training for 10 epochs
 Ep:    0, Batches: 77,  Avg Loss Tr:  1.398,     Avg Time:  0.035s 
           Batches: 44,  Avg Loss Ev:  1.118,     Avg Time:  0.011s 
                                                                    Best saved at Epoch:    0
                                                                    Epoch:    0 finished in  0.928 minutes
 Ep:    1, Batches: 77,  Avg Loss Tr:  0.962,     Avg Time:  0.025s 
           Batches: 44,  Avg Loss Ev:  0.913,     Avg Time:  0.010s 
                                                                    Best saved at Epoch:    1
 Ep:    2, Batches: 77,  Avg Loss Tr:  0.737,     Avg Time:  0.024s 
           Batches: 44,  Avg Loss Ev:  0.808,     Avg Time:  0.010s 
                                                                    Best saved at Epoch:    2
 Ep

TransformerObjectDetection(
  (encoder1): Encoder(
    (patch_embedding): PatchEmbedding(
      (ln1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (proj): Linear(in_features=192, out_features=256, bias=True)
      (activation): GELU(approximate='none')
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (blks): ModuleList(
      (0-1): 2 x Block_encoder(
        (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attention): BitMGQA(
          (fc_q): BitLinear(in_features=256, out_features=256, bias=True)
          (fc_k): BitLinear(in_features=256, out_features=128, bias=True)
          (fc_v): BitLinear(in_features=256, out_features=128, bias=True)
          (fc_o): BitLinear(in_features=128, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ffn): BitFeedForward(
      